**STACK INSTRUCTIONS** 
- R6 will store "going" addresses while going to push/pop
- R7 will store "return" addresses while going to push/pop
- R5 will be used in push/pop to point to top of stack, so won't be saved

To call a subroutine: 
-     LEA R6, [SUBROUTINE LABEL]
-     JSR PUSH

Before returning from a subroutine: 
-     LEA R0, POP
-     JMP R0

Path: 
- Store address of subroutine in R6
- JSR to push, which stores starting point in R7
- Push does JMP R6 to get to subroutine
- Subroutine JMPs to pop
- Pop RETs to starting point 

Eventually, we may need to alter some of these instructions as things get too far apart for JSR or w/e.

**ROOM/STATE DATA FORMAT**

(subject to changes?)

Every room/state stored in memory has a label pointing to the beginning of its data block, and variables stored at regular distances relative to that top. 

For all rooms:
- Label + 0: which quality (HP/stats) the room changes (0 for none) [Note: Can this field also be used to change variables, e.g. whether you have a sword?]
- Label + 1: amount of quality change
- Label + 2: type of state (0 for user-input directed, 1 for dice-directed, 2 for variable-directed)

Then if user-input room: 
- Label + 3, 4, 5: addresses of destinations for 1, 2, and 3 inputs
- Label + 6: Description string in compressed format

Or if dice room: 
- Label + 3: Stat to check
- Label + 4: Difficulty of check
- Label + 5: Destination if success
- Label + 6: Destination if failure

Or if variable room:
- Label + 3: Address of variable to check 
- Label + 4: Value of that variable that counts as a success
- Label + 5: Destination if success
- Label + 6: Destination if failure

**STAT CODES**
1. HP
2. CHA
3. VIT
4. WIS

In [3]:
.ORIG x3000
BRnzp LOOP_START

;;-----------------------------------------------------------------
;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;;-----------------------------------------------------------------
;; Main loop variables 
LOC: .FILL ROOM_UINPUT
NAME: .STRINGC "Ruby"
STATS: .FILL #0 ;; Start of stats block
HP: .FILL #0    ;; Stats + 1
CHA: .FILL #0   ;; Stats + 2
VIT: .FILL #0   ;; Stats + 3
WIS: .FILL #0   ;; Stats + 4

STAT_NAMES: .FILL #0 ;; Start of stat names block
HP_NAMEREF: .FILL HP_NAME
CHA_NAMEREF: .FILL CHA_NAME
VIT_NAMEREF: .FILL VIT_NAME
WIS_NAMEREF: .FILL WIS_NAME

HP_NAME: .STRINGC "time remaining"
CHA_NAME: .STRINGC "charisma"
VIT_NAME: .STRINGC "vitality"
WIS_NAME: .STRINGC "wisdom"

;;STAT_T1: .STRINGC "Your "
;;STAT_T2: .STRINGC " increased!\n"
;;STAT_T3: .STRINGC " decreased!\n"

;;-----------------------------------------------------------------

STATS_ADDR: .FILL STATS
STATNAMES_ADDR: .FILL STAT_NAMES

;; Main game loop
LOOP_START: 

    LD R1, LOC     ;; Load current state's address into R1 so we can access variables relative to it

    ;; Perform stat increase for room, if it has one
    
    LDR R0, R1, #0 ;; Load which_quality into R0
    BRz NO_CHANGE  ;; If which_quality is zero, skip stat change
    
    LD R5, STATS_ADDR ;; Load the address of the stat array into R5
    ADD R2, R0, R5 ;; Add the offset in R0 to the address in R5 to get the address of the appropriate stat
    LDR R3, R2, #0 ;; Load the current value of that stat into R3
    LDR R4, R1, #1 ;; Load R4 with the amount the stat will be changed
    ADD R3, R3, R4 
    STR R3, R2, #0 ;; Store modified stat back into memory
    
    ;; Print message about stat increase
    
    ;;LEA R0, STAT_T1
    ;;PUTSP
    
    LDR R2, R1, #0
    LEA R3, STAT_NAMES
    ADD R3, R3, R2
    LDR R0, R3, #0
    PUTSP

    ;;LEA R0, STAT_T2
    ;;PUTSP
    ;;LEA R0, STAT_T3
    ;;PUTSP

    ;; Todo: Skip to endgame if your HP is 0
    ;; Todo: Print name and HP - only if uinput? 
    
NO_CHANGE:
    LDR R2, R1, #2 ;; Load room_type into R2
    ADD R2, R2, #-1 
    BRn UINPUT     ;; Break to code for appropriate room type
    BRz DICE
    ;;BRp VARIABLE   

LOOP_DONE: 
    BRnzp LOOP_START   ;; Repeat main game loop
    HALT               ;; Right now this halt is never reached bc it loops forever LMAO

UINPUT:            ;; For user-directed rooms, print description, then use subroutine that reads & parses number from 1-3
    ADD R0, R1, #6 ;; Load R0 with address of room's description
    PUTSP          ;; Print room description
    
    LEA R6, CHOICE
    JSR PUSH
    BRnzp LOOP_DONE

DICE: 
    ;; Get a roll using D16
    LEA R6, D16
    JSR PUSH
    
    ;; Now there is a number between 1 and 16 in D16_roll
    LD R0, D16_roll
    
    ;; Load which_stat into R2
    LDR R2, R1, #3
    
    ;; Get the value of that stat
    LEA R3, STATS
    ADD R3, R3, R2
    LDR R3, R3, #0
    
    ;; Add it to the dice roll to get [result] + [stat bonus]
    ADD R0, R0, R3
    
    ;; Load difficulty check value into R2
    LDR R2, R1, #4
    
    ;; Negate R2 for comparison
    NOT R2, R2
    ADD R2, R2, #1
    
    ;; Add R2 to R0 to compare difficulty check to total roll
    ADD R2, R2, R0
    
    ;; Now R2 is negative if failure or zero/positive for success
    BRn FAILURE
    
    ;; If success, load address of success location into LOC
    LD R2, LOC
    ADD R2, R2, #5
    BRnzp DICE_DONE

FAILURE: 
    LD R2, LOC
    ADD R2, R2, #6

DICE_DONE: 
    LDR R3, R2, #0 ;; Load new location at address R2 into R3
    ST R3, LOC     ;; Store new location in LOC 
    BRnzp LOOP_DONE

;; Code for variable rooms....... coming soon???

;; ------------------------------------------------------------------
;; Subroutine to get a number from 1-3 from the user
;; Copied from Anita's code in printing_text

CHOICE:   
          IN
          LD R2, LOC
          LD R1, ASCII     ;; Load inverted ASCII conversion constant into R1
          ADD R0, R0, R1   ;; Add it to convert ASCII value into a number
          ADD R0, R0, #-2
          BRn CH1 ;; user chose 1
          BRz CH2 ;; user chose 2
          BRp CH3 ;; user chose 3
          
CH1:      ADD R2, R2, #3
          BRnzp CH_DONE
CH2:      ADD R2, R2, #4
          BRnzp CH_DONE
CH3:      ADD R2, R2, #5
          BRnzp CH_DONE
          
CH_DONE:  LDR R3, R2, #0 ;; Load the data at R2, aka the address of the #[choice] room, into R3
          ST LOC, R3     ;; Store that data in LOC, moving player there 
          LEA R0, POP
          JMP R0

;; Data
ASCII:   .FILL x-30

;;; -----------------------------------------------------
;;; After this runs, result in D16_roll
D16: 
    ;; Call Random, loading a new random number into X
    LEA R6, Random
    JSR Push
    
    LD R0, X
    AND R0, R0, #15
    ADD R0, R0, #1
    ST R0, D16_roll
    
    LEA R0, POP
    JMP R0
D16_roll: .FILL #0


;;; -----------------------------------------------------
;;; Memory X has next random number
Random: 
    LD R0, M
    LD R1, A
    
    LEA R6, Divide
    JSR PUSH         ; R0 / R1
    
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    
    LEA R6, Divide
    JSR PUSH        ; R0 / R1
    
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    
    LEA R6, Multiply
    JSR PUSH        
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1
    
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    
    LEA R6, Multiply
    JSR PUSH            
    LD R2, MULT_PRODUCT ; R2 <- r * TEMP2
    
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
    
TEST:  BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: ST R2, X
    LEA R0, POP         ;; Return
    JMP R0

A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0
TEMP1: .FILL #0
TEMP2: .FILL #0
BACK: .FILL #0

;;; -----------------------------------------------------
;;; Mult_Product <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:
  ST R2, MULT_PRODUCT           ; Product has the product
  LEA R0, POP
  JMP R0
  
MULT_PRODUCT: .FILL #0

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  LEA R0, POP
  JMP R0
QUOTIENT: .FILL #0
REMAINDER: .FILL #0

;; ------------------------------------------------------------------
;; Rooms data
ROOM_UINPUT: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL ROOM_UINPUT
    .FILL ROOM_RANDOM
    .FILL ROOM_UINPUT ;; Three choices all lead back here
    .STRINGC "Here we are in an example room. Enter 1 to jump in place, 2 to go to the random room, or 3 to spin in a circle\n"
    
ROOM_RANDOM: 
    .FILL #1 ;; Quality changed: HP
    .FILL #1 ;; Amount of change: 1
    .FILL #1 ;; 1: Based on dice roll
    .FILL #3 ;; Stat for dice roll: Vitality
    .FILL #8 ;; Difficulty of check
    .FILL ROOM_UINPUT ;; Destination if success
    .FILL ROOM_RANDOM ;; Destination if failure
    
ROOM_VARIABLE:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #2 ;; 2: Based on variable
    .FILL #0 ;; VAR_ADDRESS ;; What variable to check
    .FILL #0 ;; VALUE       ;; What value counts as a success
    .FILL ROOM_VARIABLE ;; Destination if success
    .FILL ROOM_VARIABLE ;; Destination if failure
    
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [ ]:
%exe

Here we are in an example room. Enter 1 to jump in place, 2 to go to the random room, or 3 to spin in a circle

Input a character> 